In [ ]:
import os
from multiprocessing import Pool

import numpy as np
import pandas as pd

from DataImporter import achar_feathers, achar_ressonancias, agrupar_por_desgaste, importar_respostas_em_frequencia

In [ ]:
PASTA_DADOS = "/home/gabrielmc/Documentos/LINSE/Atuador1"

In [ ]:
all_feathers = achar_feathers(PASTA_DADOS)
sorted_feathers = agrupar_por_desgaste(all_feathers)

In [ ]:
def import_data(feathers_catalogados: list[list[str]], imu: str) -> pd.DataFrame:
    dfs = []
    for desgaste_lvl, feathers in enumerate(feathers_catalogados):
        if feathers == [None]:
            continue # ignora níveis de desgaste sem amostras
        
        name_tags = [os.path.basename(feather)[:-8] for feather in feathers]
        inputs = [(i, imu) for i in feathers]
        with Pool() as pool:
            data = list(pool.starmap(achar_ressonancias, inputs))
        flat_data = [np.array(i)[:, :2].flatten() for i in data]
        index = pd.MultiIndex.from_product([[desgaste_lvl], name_tags], names= ["Desgaste", "Amostra"])
        dfs.append(pd.DataFrame(flat_data, index=index))
    
    return pd.concat(dfs)

In [ ]:
df_ressonancias = []
for imu in ("imu1accz", "imu2accz"):
    df_ressonancias.append(import_data(sorted_feathers, imu))

In [ ]:
df_media = (df_ressonancias[0] + df_ressonancias[1]) / 2
df_media.head()

In [ ]:
df_media.describe()

In [ ]:
df_ressonancias[0].describe()

In [ ]:
df_ressonancias[1].describe()